In [267]:
import pandas as pd
import numpy as np
import string
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import nltk as nlt
from fastapi import HTTPException


In [268]:
# Ruta del archivo
ruta = r"C:\Users\Elias\Documents\Tercer proyecto individual\Datasets_limpios\movies_credits.parquet"

# Lectura del dataset
movies_credits = pd.read_parquet(ruta)

# Convertimos el dataset en un dataframe
movies_credits = pd.DataFrame(movies_credits)

# Mostramos las primeras 5 filas del dataframe
movies_credits.head()

,budget,id,original_language,overview,popularity,production_countries,release_date,revenue,runtime,spoken_languages,...,title,vote_average,vote_count,name_collection,production_company,years,return_on_investment,genre,Director,Actor
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,Toy Story,7.7,5415.0,Toy Story Collection,Pixar Animation Studios,1995,12.451801,Animation,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,Jumanji,6.9,2413.0,No Collection,TriStar Pictures,1995,4.043035,Adventure,Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,Warner Bros.,1995,0.000000,Romance,Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,Waiting to Exhale,6.1,34.0,No Collection,Twentieth Century Fox Film Corporation,1995,5.090760,Comedy,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,Sandollar Productions,1995,0.000000,Comedy,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


In [269]:
# Descargamos el comando del lenguaje natural(NLTK)
nlt.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Elias\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [270]:
# Obtenemos las palabras vacias en el idioma ingles y español
en_stopwords = set(stopwords.words('english'))
esp_stopwords = set(stopwords.words('spanish'))

In [271]:
def limpieza_datos(cadena):
    if not isinstance(cadena, str):  # Verificamos si la entrada no es una cadena
        return ""  # Si no es una cadena (es None u otro tipo), retornamos una cadena vacía
    
    # Quitamos los caracteres especiales y convertimos a minúsculas
    cadena = re.sub(r'[^a-zA-Z ]', '', cadena)  # Usamos re.sub para eliminar caracteres especiales
    cadena = cadena.lower()  # Convertimos a minúsculas
    # Aquí puedes continuar con otras operaciones, como la tokenización
    return cadena


In [272]:
# Aplicamos la funcion a las columnas ["title"] y a la columna ["overview"]
movies_credits['title_depurado'] = movies_credits['title'].apply(limpieza_datos)
movies_credits['overview_depurado'] = movies_credits['overview'].apply(limpieza_datos)

# Visualizamos los datos
movies_credits.head()


,budget,id,original_language,overview,popularity,production_countries,release_date,revenue,runtime,spoken_languages,...,vote_count,name_collection,production_company,years,return_on_investment,genre,Director,Actor,title_depurado,overview_depurado
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,5415.0,Toy Story Collection,Pixar Animation Studios,1995,12.451801,Animation,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",toy story,led by woody andys toys live happily in his ro...
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,2413.0,No Collection,TriStar Pictures,1995,4.043035,Adventure,Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",jumanji,when siblings judy and peter discover an encha...
2,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,92.0,Grumpy Old Men Collection,Warner Bros.,1995,0.000000,Romance,Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",grumpier old men,a family wedding reignites the ancient feud be...
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,34.0,No Collection,Twentieth Century Fox Film Corporation,1995,5.090760,Comedy,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin...",waiting to exhale,cheated on mistreated and stepped on the women...
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,173.0,Father of the Bride Collection,Sandollar Productions,1995,0.000000,Comedy,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...",father of the bride part ii,just when george banks has recovered from his ...


In [273]:
# Eliminamos la columna ["overview"] , dado que tenemos la columna limpia ["overview_depurado"]
movies_credits.drop(columns=['overview'], inplace=True) 

# Convertimos las columnas necesarias a string(str)
movies_credits['title'] = movies_credits['title'].astype(str)
movies_credits['title_depurado'] = movies_credits['title_depurado'].astype(str)
movies_credits['overview_depurado'] = movies_credits['overview_depurado'].astype(str)
movies_credits['genre'] = movies_credits['genre'].astype(str)


In [274]:
# Se crea la columna "descripción combinada", que contiene los valores del título (depurado), el resumen (depurado) y el género.
movies_credits['descripcion_combinada'] = movies_credits['title_depurado'] + " " + movies_credits['overview_depurado'] + " " + movies_credits['genre']

In [275]:
# Creamos una lista de los stopwords en ambos idiomas
stopwords_en = list(en_stopwords)
stopwords_esp = list(esp_stopwords)

# Unimos los stopwords para poder realizar la vectorización
stopwords_total = stopwords_en + stopwords_esp

In [276]:
# Vectorización utilizando TF-IDF con stop words en inglés y español
# Se definen los paráemetros para convertir el texto en una matriz.
vectorizacion = TfidfVectorizer(stop_words=stopwords_total, max_features=5000) 
# Se crea una matriz TF-IDF sobre la columna "descripción combinada"
matrizTFIDF = vectorizacion.fit_transform(movies_credits['descripcion_combinada']) 
# Mostramos la dimension de la matriz
print(matrizTFIDF.shape)

(45376, 5000)


In [277]:
# Reducimos la dimension de la matriz
val_singulares = TruncatedSVD(n_components=100)
# Aplica el ajuste y la transformación a la matriz TF-IDF
matriz_sing = val_singulares.fit_transform(matrizTFIDF)
# Nuevas dimensiones sobre los valores singulares más importantes
print(matriz_sing.shape) 

(45376, 100)


In [278]:
# Dado que buscamos deployarlo y hacerlo mas ligero, se va a ajustar y reducir el dataset
df_sorted = movies_credits.sort_values(by='popularity', ascending=False)

In [279]:
# Obtenemos las 6 mil peliculas con mejor ranking
df_top = df_sorted.head(6000)

In [280]:
# Visualizamos las columnas
df_top.columns

Index(['budget', 'id', 'original_language', 'popularity',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'name_collection', 'production_company', 'years',
       'return_on_investment', 'genre', 'Director', 'Actor', 'title_depurado',
       'overview_depurado', 'descripcion_combinada'],
      dtype='object')

In [281]:
# Escogemos las columnas que no ocuparemos
columnas_out = ['spoken_languages', 'vote_average', 'popularity', 'status', 'production_countries', 'release_date', 'Director', 'Actor']

# Eliminamos las columnas
df_top.drop(columns=columnas_out, inplace=True)

# Visualizamos los datos
df_top.head()

C:\Users\Elias\AppData\Local\Temp\ipykernel_11404\1440829947.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top.drop(columns=columnas_out, inplace=True)


,budget,id,original_language,revenue,runtime,tagline,title,vote_count,name_collection,production_company,years,return_on_investment,genre,title_depurado,overview_depurado,descripcion_combinada
30665,74000000,211672,en,1.156731e+09,91.0,"Before Gru, they had a history of bad bosses",Minions,4729.0,Despicable Me Collection,Universal Pictures,2015,15.631499,Family,minions,minions stuart kevin and bob are recruited by ...,minions minions stuart kevin and bob are recru...
33316,149000000,297762,en,8.205804e+08,141.0,Power. Grace. Wisdom. Wonder.,Wonder Woman,5025.0,Wonder Woman Collection,Dune Entertainment,2017,5.507251,Action,wonder woman,an amazon princess comes to the world of man t...,wonder woman an amazon princess comes to the w...
42151,160000000,321612,en,1.262886e+09,129.0,Be our guest.,Beauty and the Beast,5530.0,No Collection,Walt Disney Pictures,2017,7.893040,Family,beauty and the beast,a liveaction adaptation of disneys version of ...,beauty and the beast a liveaction adaptation o...
43567,34000000,339403,en,2.245113e+08,113.0,All you need is one killer track.,Baby Driver,2083.0,No Collection,Big Talk Productions,2017,6.603274,Action,baby driver,after being coerced into working for a crime b...,baby driver after being coerced into working f...
24426,165000000,177572,en,6.521054e+08,102.0,From the creators of Wreck-it Ralph and Frozen,Big Hero 6,6289.0,No Collection,Walt Disney Pictures,2014,3.952154,Adventure,big hero,the special bond that develops between plussiz...,big hero the special bond that develops betwe...


In [282]:
# Vectorización utilizando TF-IDF con stop words en inglés y español
vectorizacion = TfidfVectorizer(stop_words=stopwords_total, max_features=5000) #Se definen los paráemetros para convertir el texto en una matriz.
matrizTFIDF = vectorizacion.fit_transform(df_top['descripcion_combinada']) #Se crea una matriz TF-IDF sobre la columna "descripción combinada"
print(matrizTFIDF.shape) #dimensiones de la matriz

(6000, 5000)


In [283]:
# Realizamos la reducción de dimensionalidad
val_singulares = TruncatedSVD(n_components=100) #Se reduce la dimensionalidad de la matriz a 100 dimensiones, basándose en los valores singulares más altos.
matriz_sing = val_singulares.fit_transform(matrizTFIDF)#Aplica el ajuste y la transformación a la matriz TF-IDF
print(matriz_sing.shape) #Nuevas dimensiones sobre los valores singulares más importantes

(6000, 100)


In [284]:
# Calculamos la similitud del coseno
similitud_coseno = cosine_similarity(matriz_sing, matriz_sing)

In [285]:
def recomendacion(title, df=df_top, similitud_coseno=similitud_coseno):
    try:
        indice = df_top[df_top['title'].str.lower() == title.lower()].index[0] # Se define el índice de película similar
        similitud = list(enumerate(similitud_coseno[indice])) 
        similitud = sorted(similitud, key=lambda x: x[1], reverse=True) #Se obtiene el score de similitud y se ordenan.
        # Se obtienen las películas similares, dejando afuera la primera(que es el mismo título):
        similitud = similitud[1:6]
        indices_similares = [i[0] for i in similitud]
        peliculas_similares = df['title'].iloc[indices_similares].tolist() #Se crea una lista con las películas
        mensaje = f"Se encontraron los siguientes títulos similares a '{title}':"
        print(mensaje)
        # Retornar la lista de títulos similares
        return peliculas_similares
    except IndexError:
        print(f"No se encontró la película '{title}'. Por favor, intente con otro título.")

In [ ]:
# Testeamos la función
prueba = recomendacion("Star wars")

# Visualizamos el resultado
prueba


Se encontraron los siguientes títulos similares a 'Star wars':


['You Only Live Twice',
 'S.W.A.T.: Under Siege',
 'Non-Stop',
 'Maze Runner: The Scorch Trials',
 'The Quiller Memorandum']

In [291]:
# Hacemos un 2do testeo
prueba_2 = recomendacion("The Shawshank Redemption")

# Visualizamos el resultado
prueba_2

Se encontraron los siguientes títulos similares a 'The Shawshank Redemption':


['Analyze That',
 'Asterix at the Olympic Games',
 'Good Will Hunting',
 'The Wild',
 'The Intern']

In [292]:
# Hacemos un 3er testeo
prueba_3 = recomendacion("Inception")

# Visualizamos el resultado
prueba_3

No se encontró la película 'Inception'. Por favor, intente con otro título.


In [ ]:
# Hacemos un 4to testeo
prueba_4 = recomendacion("The Godfather")

# Visualizamos el resultado
prueba_4

Se encontraron los siguientes títulos similares a 'The Godfather':


['The Scapegoat',
 'The Insider',
 'Anonymous',
 'The Passion of Joan of Arc',
 '1']

In [294]:
# Guardamos el dataframe df_top en un archivo parquet
df_top.to_parquet("recomendación.parquet")